# figure s2 vlz

- kernel: r_env, R 4.1.3
- date: 2024-03-04

## load

In [ ]:
library(tidyverse)
library(tools)
library(logging)
library(ggpubr)
library(ggsci)
library(patchwork)
library(igraph)
library(ggraph)
library(Seurat)
library(ComplexHeatmap)

source('../scripts/r_funcs.r')

theme_set(theme_pubr())
logging::basicConfig()
options(warn = -1)

outdir <- '../figures/figs2'
create_dir(outdir)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.0     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.1     ✔ tibble    3.2.0
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘igraph’


The following objects are masked from ‘package:lubridate’:

    %--%, union


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:purrr’:

    compose, simplify


The following object is masked from ‘package:tidyr’:

    crossing


The following object is masked from ‘package:tibble’:

    as_data_frame


The following objects are masked from ‘package:stats

In [2]:
f_pat_gp <- '../tables/patient_info_v2.tsv'
gp <- 'response'

gp_comp_map$response <- list(
  c('R-pre', 'R-post'),
  c('PR-pre', 'PR-post'),
  c('R-pre', 'PR-pre'),
  c('R-post', 'PR-post')
)
gp_lvls$response <- c('R', 'PR')
comb_order <- c('R-pre', 'R-post', 'PR-pre', 'PR-post')

## s2a: signature contribution

In [3]:
f_contri <- '../../stage4/a11_wes/new-muts/signature/signatue-contribution.tsv'

In [13]:
df <- read_tsv(f_contri, show_col_types = F) %>% 
    select(-c('patient', 'mandard_group', 'treatment_group')) %>%
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient_id')
df %>% head(2)

2024-03-04 13:53:12 INFO::these clinial info will be added: response


,patient_id,response,sample,sample_type,sig_name,aetiology,best_match,cosmic_name,prop,sig_cat
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,006,R,01-006-T,Baseline,Signature_8,Unknown,Best match: SBS5 [cosine-similarity: 0.745],SBS5,6.928890e-16,SBS5
2,006,R,01-006-T,Baseline,Signature_2,defective DNA mismatch repair,Best match: SBS6 [cosine-similarity: 0.608],SBS6,5.514562e-11,SBS6


In [16]:
# summarise data
dp <- summarise(df, .by = c('sample', 'sig_cat'), prop = sum(prop))

# order sample by SBS6
samp_order <- dp %>%
filter(sig_cat == 'SBS6') %>%
arrange(prop) %>%
pull(sample) %>%
unique()

# plot
p <- dp %>% 
    mutate(sig_cat = factor(sig_cat, sort(unique(sig_cat))),
           sample = factor(sample, samp_order)) %>%
    ggbarplot(x = 'sample', y = 'prop', fill = 'sig_cat', position = position_fill(), palette = 'nejm') +
    scale_y_continuous(expand = c(0, NA)) +
    labs(y = 'Signature contribution', fill = 'Signature') +
    theme(axis.text.x = element_blank(), axis.title.x = element_blank(), axis.ticks.x = element_blank(), 
          legend.title = element_text(vjust = 0.5, hjust = 0.5))
ggsave(filename = str_glue('{outdir}/figs2-bar-signature_contirbute.pdf'), plot = p, width = 7, height = 4)

## s2b: immune editing score, R-pre vs PR-pre

In [ ]:
f_edit_score <- '../../stage4/a11_wes/new-muts/immune_editing_score.tsv'
gp <- 'response'

df <- read_tsv(f_edit_score, show_col_types = F) %>% 
    filter(!grepl('-LN', sample_id), pair_cat == 'Fresh', pair_mode == 'tumor-normal-paired') %>%
    mutate(sample_type = case_match(sample_type, 'Baseline' ~ 'pre', 'Treat' ~ 'post')) %>%
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient_id')

df %>% write_tsv(str_glue('{outdir}/fig2-immune_editing_score-tn_pair-fresh-t.tsv'))
loginfo('tn_pair, fresh, nonLN: %g records, %g samples', nrow(df), length(unique(df$sample_id)))

2024-02-05 00:40:55 INFO::these clinial info will be added: response
2024-02-05 00:40:55 INFO::tn_pair, fresh, nonLN: 53 records, 53 samples


In [ ]:
# good vs poor at Baseline
p <- df %>% 
    filter(sample_type == 'pre') %>% 
    filter(.data[[gp]] %in% gp_lvls[[gp]]) %>% 
    cell_comp_boxplot(x = c(gp, 'sample_type'), y = 'editing_score', pt_fill = gp, pair_by = NULL,
                      fill_order = gp_lvls[[gp]], facet_by = NULL, xorder = c('R-pre', 'PR-pre'), xangle = 60) +
    stat_compare_means(comparisons = gp_comp_map_pre[[gp]])+
    labs(fill = '', y = 'Immune editing score') + 
    theme(legend.justification = c(1, 0))
ggsave(filename = str_glue('{outdir}/fig2e-box_edit_score-{gp}-pre-tn_pair-fresh-t.pdf'),
       plot = p, width = 2, height = 5)